# 2) Gaia RVS run selection function

* [Check fileinfo for the Gaia data](#check)
* [Generate selection function](#SF)
* [Calculate SF probabilities](#SFprob)

In [1]:
# autoreload modules when files have been altered
%load_ext autoreload
%autoreload 2

import os, sys, pickle
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

from seestar import ArrayMechanics
from seestar import SelectionGrid
from seestar import IsochroneScaling

import seaborn as sns


def ErrorWarning():
    raise ValueError("Wait, don't run this, you don't want to!")

<a name="check"></a>
# Check fileinfo

In [2]:
from seestar import surveyInfoPickler
fileinfo_path = 'PATH/GaiaRVS/GaiaRVS_fileinfo.pickle'
fileinfo = surveyInfoPickler.surveyInformation(fileinfo_path)

In [3]:
fileinfo.testFiles()

1) Checking file paths exist:
OK

2) Checking spectroscopic catalogue file structure:
OK

3) Checking field information file structure:
OK

4) Checking photometric catalogue file structure:
Checking 36.csv:
OK

5) Checking selection function pickle paths exist:
The path to your selection function pickled instance, sf_pickle_path, does not exist: /media/andy/37E3-0F91/Research/SF/SFdata/GaiaRVS/GaiaRVS_SF.pickle
The path to your selection function pickled instance, obsSF_pickle_path, does not exist: /media/andy/37E3-0F91/Research/SF/SFdata/GaiaRVS/GaiaRVS_obsSF.pickle
^ These files should exist for an already made selection function. If you're starting from scratch, ignore this!

6) Checking isochrone pickle files exist:
The path to isochrone data, iso_interp_path, does not exist: /media/andy/37E3-0F91/Research/SF/SFdata/GaiaRVS/isochrones/isochrone_interpolantinstances.pickle
The path to isochrone data, iso_data_path, does not exist: /media/andy/37E3-0F91/Research/SF/SFdata/GaiaRVS/iso

In [4]:
# You can change the model if you like (This is the default)
# 'GMM' = Gaussian mixture model, the number corresponds to the number of components
fileinfo.spectro_model = ('GMM', 1)
fileinfo.photo_model = ('GMM', 2)

In [5]:
# Make sure to save your changes if you change anything
fileinfo.save()

<a name="SF"></a>
# Calculate Selection function

In [ ]:
# Run the selection function
fileinfo_path = 'PATH/GaiaRVS/GaiaRVS_fileinfo.pickle'
SF = SelectionGrid.SFGenerator(fileinfo_path, ncores=1)
# ncores=0: SF will run the Serial version (not use multiprocessing)
# ncores>0: Parallel version on the number of cores given
# When running in parallel, the field counting can mess up a bit because the pools get disordered

Would you like the selection function in: a) observable, b) intrinsic, c) both? (return a, b or c)a
No observed SF found at /media/andy/37E3-0F91/Research/SF/SFdata/GaiaRVS/GaiaRVS_obsSF.pickle, we'll build observable seleciton function from scratch.
The spectro model description is:('GMM', 1)
The photo model description is:('GMM', 2)

{'Colmin': 'Colmin', 'Colmax': 'Colmax', 'Magmax': 'Magmax', 'fieldID': 'fieldID', 'Magmin': 'Magmin'}
Importing data for Colour-Magnitude Field interpolants...
{'Kapp': 'appB', 'Happ': 'appC', 'glon': 'phi', 'glat': 'theta', 'Japp': 'appA', 'fieldID': 'fieldID'}
Filtering for null values in spectro: Total star count = 24235. Filtered star count = 24235. 0 stars removed with null values
((1.34200000763, 12.8100004196), (-0.5150003433000006, 2.0560002326999998))
...done.

Creating Colour-Magnitude Field interpolants...
multiprocessing - observable SF calculation - running on 1 cores...
Current field in col-mag calculation: 52, 53/108, Time: 7m, Left: -3m

<a name="SFprob"></a>
# Calculate Selection function probabilities for dataset

In [ ]:
# Dataset to run selection function on
survey = pd.read_csv(fileinfo.spectro_path)
survey['Colour'] = survey[fileinfo.spectro_coords[3]] - survey[fileinfo.spectro_coords[4]]

In [ ]:
# Run the selection function on the given dataset
survey = SF(survey, method='observable', coords=['Happ', 'Colour'], angle_coords=['glon', 'glat'])
survey['union_obs'] = sub.union